In [ ]:
import torch.nn as nn

class PositionwiseFeedForward(nn.Module):
    """ A two-feed-forward-layer module """

    def __init__(self, d_in, d_hid, kernel_size, dropout=0.1):
        super().__init__()

        # Use Conv1D
        # position-wise
        self.w_1 = nn.Conv1d(
            d_in,
            d_hid,
            kernel_size=kernel_size[0],
            padding=(kernel_size[0] - 1) // 2,
            groups=16,
        )
        
        self.layer_norm = nn.LayerNorm(d_in)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        residual = x
        output = x.transpose(1, 2)
        output = self.w_2(F.relu(self.w_1(output)))
        output = output.transpose(1, 2)
        output = self.dropout(output)
        output = self.layer_norm(output + residual)

        return output
l = PositionwiseFeedForward(768,2048,[21,1])
l , sum([p.numel() for p in l.parameters()]) / 1e6

(PositionwiseFeedForward(
   (w_1): Conv1d(768, 2048, kernel_size=(21,), stride=(1,), padding=(10,), groups=16)
   (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
   (dropout): Dropout(p=0.1, inplace=False)
 ),
 2.067968)

: 